- Since time.clock() is no longer supported in python 3.8, clock() has been replaced to perf_counter() in three files of openopt: result.py, runProbSolver.py, ooIter.py
- Solver for linear optimization has changed to gurobi in lpsolver.py 

In [ ]:
from os import makedirs
import pandas as pd
import numpy as np
from freeflux import Model

merged_dict = {
    
    'mmo_mdh' : ['rxn00843_c0', 'rxn14207_c0'], # 2
    'H4MPT_nadh'  : ['rxn07847_c0', 'rxn37236_c0', 'rxn02480_c0', 'rxn02431_c0', 'rxn07849_c0'], # 5
    'H4MPT_nadph' : ['rxn07847_c0', 'rxn07848_c0', 'rxn02480_c0', 'rxn02431_c0', 'rxn07849_c0'], # 5
    'H4F_reductive' : ['rxn01211_c0', 'rxn00907_c0'], # 2
    'ser_to_pg_nadh' : ['rxn00424_c0', 'rxn01011_c0', 'rxn01102_c0'], # 3
    'ser_to_pg_nadph' : ['rxn00424_c0', 'rxn01013_c0', 'rxn01102_c0'], # 3
    'pg_to_pep' : ['rxn01106_c0', 'rxn00459_c0'], # 2
    'mal_to_glx_accoa' : ['rxn00934_c0', 'rxn00331_c0'], # 2
    'EMC_nadh' : ['rxn00178_c0', 'rxn01451_c0', 'rxn02345_c0', 'rxn02167_c0', 'rxn16150_c0', 'rxn16825_c0', 
                  'rxn16151_c0', 'rxn25269_c0', 'rxn03440_c0', 'rxn00682_c0', 'rxn01355_c0', 'rxn01996_c0', 'rxn00602_c0'], # 13
    'EMC_nadph' : ['rxn00178_c0', 'rxn01453_c0', 'rxn02345_c0', 'rxn02167_c0', 'rxn16150_c0', 'rxn16825_c0', 
                  'rxn16151_c0', 'rxn25269_c0', 'rxn03440_c0', 'rxn00682_c0', 'rxn01355_c0', 'rxn01996_c0', 'rxn00602_c0'], # 13
    'cit_to_akg' : ['rxn00974_c0', 'rxn01388_c0', 'rxn00198_c0'], # 3
    'pg_to_gap' : ['rxn01100_c0', 'rxn00781_c0'], # 2
    'f6p_to_gap_dhap' : ['rxn00786_c0', 'rxn00549_c0', 'rxn00747_c0'], # 3
    'x5p_r5p' : ['rxn00777_c0', 'rxn01116_c0'], # 2
    'e4p_dhap_to_s7p' : ['rxn01334_c0', 'rxn01343_c0'] # 2
# total 
}



DILUTION_FROM = [
    'cpd00011u', # co2
    'cpd00023u', # glutamate
    'cpd00024u' # akg
    # 'cpd00041u' # aspartate
    # 'cpd00036u', # succinate
    # 'cpd00072u', # f6p
    # 'cpd00101u', # r5p
    # 'cpd00106u', # fumarate
    # 'cpd00130u', # malate
    # 'cpd00137u' # citrate
    # 'cpd00169u', # phosphoglycerate
    # 'cpd00236u', # e4p
    # 'cpd00238u'  # s7p
]


# estimate fluxes at steady state
def steady_state_fitting():

    ob3b = Model('OB3b_wt')
    ob3b.read_from_file(MODEL_FILE)
    
    with ob3b.fitter('ss') as fit:
        # specify the lableing strategy, 
        # use this method for every labeled substrate
        fit.set_labeling_strategy(
            'cpd01024.ex', 
            labeling_pattern = ['1'], 
            percentage = [0.25], 
            purity = [0.99]
        )
        
        # read measurements
        fit.set_measured_MDVs_from_file(MEASURED_MDVS)
        fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
        
        # set upper and lower bounds for fluxes
        fit.set_flux_bounds('all', bounds = [-100, 100]) 
        
        # solve the fluxes
        fit.prepare(
            dilution_from = DILUTION_FROM, 
            n_jobs = 30
        )
        while True:
            res = fit.solve(solver = 'ralg', max_iters = 1000)
            if res.optimization_successful:
            # if res.opt_objective < 2000:
                break
    
    # save the results
    pd.Series(res.opt_net_fluxes).to_excel(
        OUT_DIR+'/estimated_net_fluxes.xlsx'
    )
    pd.Series(res.opt_total_fluxes).to_excel(
        OUT_DIR+'/estimated_total_fluxes.xlsx'
    )

    net_cis = res.estimate_confidence_intervals(
        which = 'net', 
        confidence_level = 0.95
    )
    total_cis = res.estimate_confidence_intervals(
        which = 'total', 
        confidence_level = 0.95
    )
    pd.DataFrame(net_cis, index = ['LB', 'UB']).T.to_excel(
        OUT_DIR+'/netflux_le_CIs.xlsx'
    )
    pd.DataFrame(total_cis, index = ['LB', 'UB']).T.to_excel(
        OUT_DIR+'/totalflux_le_CIs.xlsx'
    )

    # normal probability plot of residuals
    res.plot_normal_probability(show_fig = False, output_dir = OUT_DIR)
    
    # compare simulations and measurements
    res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = OUT_DIR)
    res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = OUT_DIR)
    
    # export the contribution matrix
    res.estimate_contribution_matrix(which = 'net').to_excel(
        OUT_DIR+'/netflux_contribMat.xlsx'
    )
    res.estimate_contribution_matrix(which = 'total').to_excel(
        OUT_DIR+'/totalflux_contribMat.xlsx'
    )
    
    # export the sensitivity matrix
    res.estimate_sensitivity(which = 'net').to_excel(
        OUT_DIR+'/netflux_senMat.xlsx'
    )
    res.estimate_sensitivity(which = 'total').to_excel(
        OUT_DIR+'/totalflux_senMat.xlsx'
    )

    df = pd.DataFrame({'opt_resids': res.opt_resids})
    df.to_csv(OUT_DIR + "/opt_resids.csv", index=False)

    sim_flux = pd.Series(res.opt_net_fluxes)
    sim_flux.to_csv(
    OUT_DIR+'/estimated_net_fluxes.csv'
    )
    
    flux = {}
    merged_rxn = list(merged_dict.keys())
    for i in range(len(sim_flux)):
        rxn_id = sim_flux.index[i]
        rxn_flux = sim_flux.values[i]
        
        if rxn_id in merged_rxn:
            rxns = merged_dict[rxn_id]
            for r in rxns:
                if r not in flux:
                    flux[r] = rxn_flux
                else:
                    flux[r] += rxn_flux
        else:
            flux[rxn_id] = rxn_flux
    flux_pd_series = pd.Series(flux)
    flux_pd_series.to_csv(OUT_DIR + 'net_fluxes.csv')
    
    ref = flux['rxn00692_c0']
    norm_flux = pd.Series(
        list(flux.values())/ref, name = 'fluxes', index = list(flux.keys())
    )
    norm_flux.to_csv(OUT_DIR + 'net_normalized_fluxes.csv')

    
    data_list = []
    for key in res.simulated_MDVs.keys():
        for i in range(len(res.simulated_MDVs[key])):
            data_list.append([key, f"M{i}", "Simulated", res.simulated_MDVs[key][i], None])  # No SD for simulated
            data_list.append([key, f"M{i}", "Measured", res.measured_MDVs[key][0][i], res.measured_MDVs[key][1][i]])  # With SD
    
    df = pd.DataFrame(data_list, columns=["Metabolite", "Isotopomer", "Type", "Value", "SD"])
    df.to_csv(OUT_DIR + "/MDVs_data.csv", index=False)
    
    return res


In [ ]:
base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM_OB3b/multi_omics/metabolomic/13c_mfa/'
MODEL_FILE       = base_dir + 'FREEFLUX_MODEL.xlsx'
OUT_DIR          = base_dir

# WT
MEASURED_MDVS    = base_dir + 'FREEFLUX_MDV_WT.xlsx'
MEASURED_FLUXES  = base_dir + 'FREEFLUX_FLUX_WT.xlsx'
wt_mfa           = steady_state_fitting()

# ΔargE::PmxaFargJ
MEASURED_MDVS    = base_dir + 'FREEFLUX_MDV_MUT_1.xlsx'
MEASURED_FLUXES  = base_dir + 'FREEFLUX_FLUX_MUT_1.xlsx'
mut_1_mfa        = steady_state_fitting()

# WT + pTJS140-PmxaFargJ
MEASURED_MDVS    = base_dir + 'FREEFLUX_MDV_MUT_2.xlsx'
MEASURED_FLUXES  = base_dir + 'FREEFLUX_FLUX_MUT_2.xlsx'
mut_2_mfa        = steady_state_fitting()

